In [86]:
import pandas as pd
import pyodbc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy
from sqlalchemy import create_engine


In [87]:
conn = pyodbc.connect(
       driver = "ODBC Driver 17 for SQL Server",
       server = "Shoto",
       database = "Aviva_plc",
       trusted_connection = "yes")
            
cursor = conn.cursor()
cursor.execute("SELECT * from [IS]")
                
rows = cursor.fetchall()
                
# Get column names from cursor.description
columns = [column[0] for column in cursor.description]
# Create a DataFrame
IS = pd.DataFrame.from_records(rows, columns=columns)
new_header = IS.iloc[0]  # Grab the first row for the header
IS = IS[1:]  # Take the data less the header row,
IS.columns = new_header  # Set the header row as the IS header
print(IS)

engine = create_engine('mssql+pyodbc://Shoto/Aviva_plc?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes')

# Step 3: Save DataFrame to SQL
IS.to_sql('cleaned_IS', engine, if_exists='replace', index=False)

print("DataFrame saved to SQL database successfully.")


0                     Fiscal data as of Dec 31 2023   2023   2022   2021
1                          REVENUE AND GROSS PROFIT   None   None   None
2                                     Total revenue  20145  14382  32638
3                                OPERATING EXPENSES   None   None   None
4                             Cost of revenue total    --     --     -- 
5       Selling, general and admin. expenses, total   1054    953   4102
6                         Depreciation/amortization    244    267    409
7                           Unusual expense(income)      3      7    -21
8                   Other operating expenses, total    458   -581  14351
9                           Total operating expense  18352  16520  31761
10                                 Operating income   1793  -2138    877
11                                       Other, net   -103   -101    -76
12  INCOME TAXES, MINORITY INTEREST AND EXTRA ITEMS   None   None   None
13                          Net income before taxes

DataFrame saved to SQL database successfully.


In [88]:
cursor.execute("SELECT * from [CF]")
rows = cursor.fetchall()

# Get column names from cursor.description
columns = [column[0] for column in cursor.description]

# Create a DataFrame
CF = pd.DataFrame.from_records(rows, columns=columns)
new_header = CF.iloc[0]  # Grab the first row for the header
CF = CF[1:]  # Take the data less the header row
CF.columns = new_header

engine = create_engine('mssql+pyodbc://Shoto/Aviva_plc?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes')
        
# Step 3: Save DataFrame to SQL
CF.to_sql('cleaned_CF', engine, if_exists='replace', index=False)

print("DataFrame saved to SQL database successfully.")
print(CF)

DataFrame saved to SQL database successfully.
0                Fiscal data as of Dec 31 2023   2023   2022   2021
1                                   OPERATIONS   None   None   None
2                                   Net income   1690  -2379    801
3                       Depreciation/depletion     67     57     74
4                               Non-Cash items    961    324   4149
5                Cash taxes paid, supplemental     68    210    304
6             Cash interest paid, supplemental    206    450    489
7                   Changes in working capital  -5450  17881  -4804
8                   Total cash from operations  -2732  15883    220
9                                    INVESTING   None   None   None
10                        Capital expenditures   -350    -99   -108
11  Other investing and cash flow items, total   -      -240    182
12                   Total cash from investing   -350   -339     74
13                                   FINANCING   None   None   None
14

In [89]:
cursor.execute("SELECT * from [BS]")
rows = cursor.fetchall()
# Get column names from cursor.description
columns = [column[0] for column in cursor.description]

# Create a DataFrame\r\n",
BS = pd.DataFrame.from_records(rows, columns=columns)
new_header = BS.iloc[0]  # Grab the first row for the header
BS = BS[1:]  # Take the data less the header row
BS.columns = new_header

engine = create_engine('mssql+pyodbc://Shoto/Aviva_plc?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes')
     
# Step 3: Save DataFrame to SQL
BS.to_sql('cleaned_BS', engine, if_exists='replace', index=False)
            
print("DataFrame saved to SQL database successfully.")
print(BS)

DataFrame saved to SQL database successfully.
0                   Fiscal data as of Dec 31 2023    2023    2022    2021
1                                          ASSETS    None    None    None
2                 Cash And Short Term Investments      --      --      --
3                          Total Receivables, Net    3721    3480    3740
4                                 Total Inventory     --      --      -- 
5                                Prepaid expenses     --      --      -- 
6                     Other current assets, total     --      --      -- 
7                            Total current assets     --      --      -- 
8                Property, plant & equipment, net     424     350     428
9                                   Goodwill, net    2100    2102    1741
10                               Intangibles, net     968     940     994
11                          Long term investments  281105  256615  312477
12                    Note receivable - long term      95     336 

# **_Financial Ratio Analysis_**

In [90]:
print("Income Statement 2023")
IS2023 = pd.read_sql_query('select [Fiscal data as of Dec 31 2023], [2023] from [cleaned_IS]', engine)
print(IS2023)

print("Cash Flow Statement 2023")
CF2023 = pd.read_sql_query('select [Fiscal data as of Dec 31 2023], [2023] from [cleaned_CF]', engine)
print(CF2023)

print("Balance Sheet 2023")
BS2023 = pd.read_sql_query('select [Fiscal data as of Dec 31 2023], [2023] from [cleaned_BS]', engine)
print(BS2023)

Income Statement 2023


                      Fiscal data as of Dec 31 2023   2023
0                          REVENUE AND GROSS PROFIT   None
1                                     Total revenue  20145
2                                OPERATING EXPENSES   None
3                             Cost of revenue total    -- 
4       Selling, general and admin. expenses, total   1054
5                         Depreciation/amortization    244
6                           Unusual expense(income)      3
7                   Other operating expenses, total    458
8                           Total operating expense  18352
9                                  Operating income   1793
10                                       Other, net   -103
11  INCOME TAXES, MINORITY INTEREST AND EXTRA ITEMS   None
12                          Net income before taxes   1690
13                       Provision for income taxes    584
14                           Net income after taxes   1106
15                                Minority interest    -

## **_Profitability Ratios_**

### **Return on Asset**

In [91]:
NI1 = pd.read_sql("select [2023] from [cleaned_IS] WHERE [Fiscal data as of Dec 31 2023] = 'Net income'", engine)

TA1 = pd.read_sql("select [2023] from [cleaned_BS] where [Fiscal data as of Dec 31 2023] = 'Total assets'", engine)

net_income1 = pd.to_numeric(NI1.iloc[0, 0])
total_assets1 = pd.to_numeric(TA1.iloc[0, 0])

ROA_value1 = (net_income1 / total_assets1)*100

ROA1 = round(ROA_value1,2)

print("Return on Asset 2023 = ",ROA1,"%" )

#ROA of 2022
NI2 = pd.read_sql("select [2022] from [cleaned_IS] WHERE [Fiscal data as of Dec 31 2023] = 'Net income'", engine)

TA2 = pd.read_sql("select [2022] from [cleaned_BS] where [Fiscal data as of Dec 31 2023] = 'Total assets'", engine)

net_income2 = pd.to_numeric(NI2.iloc[0, 0])
total_assets2 = pd.to_numeric(TA2.iloc[0, 0])

ROA_value2 = (net_income2 / total_assets2)*100

ROA2 = round(ROA_value2,2)

print("Return on Asset 2022 = ",ROA2,"%" )

#ROA of 2021
NI3 = pd.read_sql("select [2021] from [cleaned_IS] WHERE [Fiscal data as of Dec 31 2023] = 'Net income'", engine)

TA3 = pd.read_sql("select [2021] from [cleaned_BS] where [Fiscal data as of Dec 31 2023] = 'Total assets'", engine)

net_income3 = pd.to_numeric(NI3.iloc[0, 0])
total_assets3 = pd.to_numeric(TA3.iloc[0, 0])

ROA_value3 = (net_income3 / total_assets3)*100

ROA3 = round(ROA_value3,2)

print("Return on Asset 2021 = ",ROA3,"%" )

Return on Asset 2023 =  0.33 %
Return on Asset 2022 =  -0.34 %
Return on Asset 2021 =  0.56 %


### **Net Profit Margin**

In [92]:
TR1 = pd.read_sql("select [2023] from [cleaned_IS] WHERE [Fiscal data as of Dec 31 2023] = 'Total revenue'", engine)
NetI1 = pd.read_sql("select [2023] from [cleaned_IS] WHERE [Fiscal data as of Dec 31 2023] = 'Net income'", engine)
total_revenue1 = pd.to_numeric(TR1.iloc[0, 0])
net_incom1 = pd.to_numeric(NetI1.iloc[0, 0])           
NPM_value1 = (net_income1 / total_revenue1)*100
NPM1 = round(NPM_value1,2)

print("Net Profit Margin 2023 = ",NPM1,"%" )

TR2 = pd.read_sql("select [2022] from [cleaned_IS] WHERE [Fiscal data as of Dec 31 2023] = 'Total revenue'", engine)
NI2 = pd.read_sql("select [2022] from [cleaned_IS] WHERE [Fiscal data as of Dec 31 2023] = 'Net income'", engine)
total_revenue2 = pd.to_numeric(TR2.iloc[0, 0])
netincome1 = pd.to_numeric(NI2.iloc[0, 0])           
NPM_value2 = (netincome1 / total_revenue2)*100
NPM2 = round(NPM_value2,2)

print("Net Profit Margin 2022 = ",NPM2,"%" )

TR3 = pd.read_sql("select [2021] from [cleaned_IS] WHERE [Fiscal data as of Dec 31 2023] = 'Total revenue'", engine)
NI3 = pd.read_sql("select [2021] from [cleaned_IS] WHERE [Fiscal data as of Dec 31 2023] = 'Net income'", engine)
total_revenue3 = pd.to_numeric(TR3.iloc[0, 0])
net_income3 = pd.to_numeric(NI3.iloc[0, 0])           
NPM_value3 = (net_income3 / total_revenue3)*100
NPM3 = round(NPM_value3,2)

print("Net Profit Margin 2021 = ",NPM3,"%" )


Net Profit Margin 2023 =  5.39 %
Net Profit Margin 2022 = 

 -7.31 %
Net Profit Margin 2021 =  6.02 %


### **Return on Equity**

In [93]:
#ROE for 2023
SE1 = pd.read_sql("select [2023] from [cleaned_BS] WHERE [Fiscal data as of Dec 31 2023] = 'Total equity'", engine)
Net1 = pd.read_sql("select [2023] from [cleaned_IS] WHERE [Fiscal data as of Dec 31 2023] = 'Net income'", engine)
shareholders_equity1 = pd.to_numeric(SE1.iloc[0,0])
netin1 = pd.to_numeric(Net1.iloc[0, 0])
ROE_value1 = (netin1 / shareholders_equity1) * 100
ROE1 = round(ROE_value1,2)

print("Retrun on Equity 2023 = ",ROE1,"%")

#ROE for 2022
SE2 = pd.read_sql("select [2022] from [cleaned_BS] WHERE [Fiscal data as of Dec 31 2023] = 'Total equity'", engine)
Net2 = pd.read_sql("select [2022] from [cleaned_IS] WHERE [Fiscal data as of Dec 31 2023] = 'Net income'", engine)
shareholders_equity2 = pd.to_numeric(SE2.iloc[0,0])
netin2 = pd.to_numeric(Net2.iloc[0, 0])
ROE_value2 = (netin2 / shareholders_equity2) * 100
ROE2 = round(ROE_value2,2)

print("Retrun on Equity 2022 = ",ROE2,"%")

# ROE for 2021
SE3 = pd.read_sql("select [2021] from [cleaned_BS] WHERE [Fiscal data as of Dec 31 2023] = 'Total equity'", engine)
Net1 = pd.read_sql("select [2021] from [cleaned_IS] WHERE [Fiscal data as of Dec 31 2023] = 'Net income'", engine)
shareholders_equity3 = pd.to_numeric(SE3.iloc[0,0])
netin3 = pd.to_numeric(Net1.iloc[0, 0])
ROE_value3 = (netin3 / shareholders_equity3) * 100
ROE3 = round(ROE_value3,2)

print("Retrun on Equity 2021 = ",ROE3,"%")

Retrun on Equity 2023 =  11.69 %
Retrun on Equity 2022 =  -10.61 %
Retrun on Equity 2021 = 

 11.96 %


## **_Solvency Ratios_**

### **Debt to Asset Ratio**

In [94]:
#DoA of 2023
TL1 = pd.read_sql("select [2023] from [cleaned_BS] WHERE [Fiscal data as of Dec 31 2023] = 'Total liabilities'", engine)
TotA1 = pd.read_sql("select [2023] from [cleaned_BS] where [Fiscal data as of Dec 31 2023] = 'Total assets'", engine)
total_liabilities1 = pd.to_numeric(TL1.iloc[0, 0])
total_asset1 = pd.to_numeric(TotA1.iloc[0, 0])
DR_value1 = (total_liabilities1 / total_asset1)*100
DR1 = round(DR_value1,2)

print("Debt to Asset Ratio 2023 = ",DR1,"%" )

#DoA of 2022
TL2 = pd.read_sql("select [2022] from [cleaned_BS] WHERE [Fiscal data as of Dec 31 2023] = 'Total liabilities'", engine)
TotA2 = pd.read_sql("select [2022] from [cleaned_BS] where [Fiscal data as of Dec 31 2023] = 'Total assets'", engine)
total_liabilities2 = pd.to_numeric(TL2.iloc[0, 0])
total_asset2 = pd.to_numeric(TotA2.iloc[0, 0])
DR_value2 = (total_liabilities2 / total_asset2)*100
DR2 = round(DR_value2,2)

print("Debt to Asset Ratio 2022 = ",DR2,"%" )

#DoA of 2021
TL3 = pd.read_sql("select [2021] from [cleaned_BS] WHERE [Fiscal data as of Dec 31 2023] = 'Total liabilities'", engine)
TotA3 = pd.read_sql("select [2021] from [cleaned_BS] where [Fiscal data as of Dec 31 2023] = 'Total assets'", engine)
total_liabilities3 = pd.to_numeric(TL3.iloc[0, 0])
total_asset3 = pd.to_numeric(TotA3.iloc[0, 0])
DR_value3 = (total_liabilities3 / total_asset3)*100
DR3 = round(DR_value3,2)

print("Debt to Asset Ratio 2021 = ",DR3,"%" )

Debt to Asset Ratio 2023 =  97.18 %
Debt to Asset Ratio 2022 =  96.8 %
Debt to Asset Ratio 2021 =  95.33 %


### **Debt to Equity Ratio**

In [95]:
#DoE of 2023
DE_value1 = (total_liabilities1 / shareholders_equity1)
DE1 = round(DE_value1,2)

print("Debt to Equity Ratio 2023 = ",DE1 )

#DoE of 2022
DE_value2 = (total_liabilities2 / shareholders_equity2)
DE2 = round(DE_value2,2)

print("Debt to Equity Ratio 2022 = ",DE2 )

#DoE of 2021
DE_value3 = (total_liabilities3 / shareholders_equity3)
DE3 = round(DE_value3,2)

print("Debt to Equity Ratio 2021 = ",DE3 )

Debt to Equity Ratio 2023 =  34.43
Debt to Equity Ratio 2022 =  30.26
Debt to Equity Ratio 2021 =  20.41


## _**Investment Ratios**_

In [96]:
EPS1 = int(40)
EPS2 = int(60)
EPS3 = int(43)

PriceperShare1 = int(434.70)
PriceperShare2 = int(442.80)
PriceperShare3 = int(405.48)

PE1 = round(EPS1 / PriceperShare1, 2)
print(PE1)

PE2 = round(EPS2 / PriceperShare2, 2)
print(PE2)

PE3 = round(EPS3 / PriceperShare3, 2)
print(PE3)

Dividend_per_Share1 = int(20.70)
Dividend_per_Share2 = int(14.70)
Dividend_per_Share3 = int(14.00)

#Payout Ratio
Payoutratio1 = round(EPS1 / Dividend_per_Share1, 2)
Payoutratio2 = round(EPS2 / Dividend_per_Share2, 2)
Payoutratio3 = round(EPS3 / Dividend_per_Share3, 2)
print(Payoutratio1)

#Dividend Yield
DY1 = round(Dividend_per_Share1 / PriceperShare1, 2)
DY2 = round(Dividend_per_Share2 / PriceperShare2, 2)
DY3 = round(Dividend_per_Share3 / PriceperShare3, 2)
print(DY3)


0.09
0.14
0.11
2.0
0.03


In [97]:
#Creating DataFrame
ratios_df = pd.DataFrame({
    'Year': [2023, 2022, 2021],
    'Return on Asset': [f"{ROA1}%", f"{ROA2}%", f"{ROA3}%"],
    'Net Profit Margin': [f"{NPM1}%", f"{NPM2}%", f"{NPM3}%"],
    'Return on Equity': [f"{ROE1}%", f"{ROE2}%", f"{ROE3}%"],
    'Debt to Asset': [f"{DR1}%", f"{DR2}%", f"{DR3}%"],
    'Debt to Equity': [f"{DE1}%", f"{DE2}%", f"{DE3}%"],
    'Earnings per Share' : [EPS1,EPS2,EPS3],
    'Price to Equity Ratio' : [PE1,PE2,PE3],
    'Dividend per Share' : [Dividend_per_Share1, Dividend_per_Share2, Dividend_per_Share3],
    'Payout Ratio' : [f"{Payoutratio1}%", f"{Payoutratio2}%",f"{Payoutratio3}%"],
    'Stock Price per Share' : [PriceperShare1, PriceperShare2, PriceperShare3],
    'Dividend Yield' : [f"{DY1}%", f"{DY2}%", f"{DY3}%"]

})

    
print(ratios_df)

#Save DataFrame to SQL
ratios_df.to_sql('Fin_ratios_Aviva', engine, if_exists='replace', index=False)
            
print("DataFrame saved to SQL database successfully.")

   Year Return on Asset Net Profit Margin Return on Equity Debt to Asset  \
0  2023           0.33%             5.39%           11.69%        97.18%   
1  2022          -0.34%            -7.31%          -10.61%         96.8%   
2  2021           0.56%             6.02%           11.96%        95.33%   

  Debt to Equity  Earnings per Share  Price to Equity Ratio  \
0         34.43%                  40                   0.09   
1         30.26%                  60                   0.14   
2         20.41%                  43                   0.11   

   Dividend per Share Payout Ratio  Stock Price per Share Dividend Yield  
0                  20         2.0%                    434          0.05%  
1                  14        4.29%                    442          0.03%  
2                  14        3.07%                    405          0.03%  


DataFrame saved to SQL database successfully.


###